#### Física de la Radioterapia. Master de Física Biomédica. Universidad Complutense de Madrid
# Calibración del acelerador lineal
## Haces de fotones medidos en condiciones de referencia
-----

**Objetivos**: 
Determinar la dosis absorbida producida por un campo de radiación en condiciones estándar de calibración utilizando una cámara de ionización tipo Farmer y aplicando el [protocolo de dosimetría de la IAEA TRS398](https://www-pub.iaea.org/MTCD/publications/PDF/TRS_398s_Web.pdf). Se recomienda leer al menos el epígrafe **3. FORMALISMO BASADO EN** $N_{D,w}$ del protocolo.

**Antecendentes**: De acuerdo al TRS398 la magnitud $D_{w, Q}$, dosis en agua para la calidad de la radiación $Q$, se puede determinar mediante la expresión

\begin{equation}
    D_{w, Q} = M_Q\;N_{D, w, Q_0}\;k_{Q, Q_0}\;k_{sat}\;k_{pol}.
\end{equation}

donde:
- $M_Q$ es la lectura neta del electrómetro expresado en condiciones normales
- $N_{D, w, Q_0}$ es el valor del factor de calibración de la cámara en condiciones de referencia
- $k_{Q, Q_0}$ es el factor de corrección por la calidad del haz del factor de calibración
- $k_{sat}$ es el factor de corrección por saturación de la cámara
- $k_{pol}$ es factor de corrección por dependencia de la lectura de la cámara con la polarización de sus electrodos

**Datos**:
Los datos de la cámara empleada están recogidos en su [certificado de calibración](https://raw.githubusercontent.com/csarux/TutorialesFisicaRadioterapia/main/Enunciados/CalibracionAcelerador/docs/CamaraFarmer10820.pdf)

Previamente se ha determinado que para esta cámara los factores de corrección por saturación $k_{sat}$ y por polarización $k_{pol}$ son ambos igual a uno en las condiciones de medida consideradas.

La energía del haz empleado corresponde a una razón TPR${}_{20, 10} = 0.694$.

Para recolectar la carga producida en la cámara se utiliza un electrómetro con una resolución de $0.001$ nC.

## Cuestión
1. Obtener de los datos del TRS398 el factor de corrección de la calibración de la cámara por la energía para el haz utilizado.

**Ayuda**: Consultar el CUADRO 14 página 85.

## Módulo `simcaliblinac`
Este cuaderno simula las condiciones variables de la medida mediante generadores aleatorios que siguen distribuciones realistas. Para realizar el ejercicio cada alumno generará y recogerá tanto las medidas como las condiciones en las que las ha realizado.

El módulo `simcaliblinac` contiene las funciones para realizar esta simulación.

```
Nota: El código de este modulo se considera aún en desarrollo.
Si al instalar el código aparecen mensajes de error o no es posible localizarlo escribir a cesaro02@ucm.es  
```

En la siguiente celda instalamos el paquete e importamos el módulo

In [ ]:
%pip install --index-url https://test.pypi.org/simple/ simcaliblinac
import simcaliblinac.simcaliblinac as scl

Tomar los valores de la presión atmosférica indicada por el barómetro del Servicio y de la temperatura medida por la sonda del termómetro que introducimos en el agua. 

Las unidades de las medidas son presión en hPa y temperatura en grados centígrados.

In [ ]:
presion, temperatura = scl.presion_temperatura_medida()
presion, temperatura

El módulo `simcaliblinac` ofrece una función para calcular la corrección por presión y temperatura $\phi$pT

In [ ]:
ϕpT = scl.ϕpTf(presion, temperatura)
ϕpT

Una vez montados los equipos, fijadas las condiciones experimentales y con todas las conexiones realizadas, comenzamos por medir la lecutra del electrómetro cuando no hay radiación. En Radioterapia a este valor lo denominamos *corriente de fugas*. Es el equivalente a una lectura de un valor de fondo pero de origen diferente.

Las fugas son de naturaleza aleatoria. Su valor medio se estima integrando durante un tiempo dado suficientemente largo como para promediar variaciones instantáneas. 

El módulo `simcaliblinac` expone un método para obtener una muestra de las fugas recolectadas durante un tiempo dado

In [ ]:
fugas = scl.fugas(minutos=2)
fugas # en nanoCulombios

Para hacer las medidas en el acelerador tenemos que fijar un número de unidades monitor nominales.

In [ ]:
UM_nominales = 200

El módulo `simcaliblinac` expone un método para obtener una muestra de **n** medidas con las unidades monitor fijadas. La tasa de UM fijada en el acelerador es de 600 UM/min. Usualmente se toma una muestra de tamaño tres, porque es la menor muestra que permite tomar una decisión sobre la coherencia de las medidas. Tres medidas permiten asegurar que no ha ocurrido nada anómalo al tomar los datos si ningún valor discrepa mucho de los otros dos.

In [ ]:
lecturas = scl.lectura_medida_f(UM=UM_nominales, n=3)
lecturas # en nanoCulombios

Para calibrar el acelerador expresamos la medida de dosis en términos relativos a las unidades monitor, es decir, queremos saber cuánta dosis damos por unidad monitor. Por una decisión complemtamente arbitraria se suele ajustar la calibración para que en condiciones estándar una UM coincida con un centiGray. Este ajuste se realiza variando la ganancia de la cámaras monitoras. El cambio de la ganancia permite variar la lectura de la cámara aún recibiendo la misma dosis.

## Cuestiones
2. Determinar la dosis absorbida suministrada por el acelerador y el output.
1. Hacer una estimación de la incertidumbre en la medida de la dosis absorbida y en la medida del output

## Comprobaciones y simulaciones estadísticas

En el Servicio de Radioterapia en el que estamos realizando estas medidas se tiene el criterio de recalibrar el acelerador, ajustar de nuevo las ganancias de las cámaras, si la variación del output respecto al valor nominal de 1 es mayor del 1.5%.

En la teoría del control de calidad se distinguen dos conceptos: *tolerancia* e *incertidumbre*.
- **Tolerancia**: Rango de la variable de control que daría lugar a resultados aceptables. Por ejemplo se consideran tolerables variaciones en la dosis final que reciben los pacientes inferiores al 5% porque no ha sido posible detectar diferencias clínicas en los resultados de tratamientos realizados con esas variaciones.

- **Incertidumbre**: Rango en el que se espera se encuentre el valor *real* de una magnitud medida.

En general se requiere que la variable de control se mida con una incertidumbre inferior a la tolerancia, idealmente incluso despreciable.

El tratamiento combinado de  la tolerancia y la incertidumbre lleva al concepto de *nivel de acción*, como el umbral para el resultado del control que indica la necesidad de realizar alguna corrección.

Dependiendo cuánto de crítico sea el resultado de nuestro control se definen dos posibles formas de situar el nivel de acción:
- **Nivel de acción mayor que la tolerancia**: Se aumenta la tolerancia con la incertidumbre. Expandimos la tolerancia porque aceptamos que los resultados puedan salirse de tolerancia si hay una cierta probabilidad de que estén dentro de tolerancia por su incertidumbre. Por ejemplo si los radares de la DGT miden la velocidad con una incertidumbre del 4% (k=3) solo nos multan si miden que nuestra velocidad supera el límite señalado en más de un 4%. En un tramo a  120 km/h multan si miden más de 124.8 km/h porque de esa manera el 99% de los multados realmente iban a más de 120. Solo hay un 1% de probabilidad de recibir una multa injusta.
- **Nivel de acción inferior la tolerancia**: Se reduce la tolerancia con la incertidumbre. Disminuimos la tolerancia porque no aceptamos los resultados que aunque se encuentren dentro de tolerancia tienen una cierta probabilidad de indicar un resultado fuera de tolerancia si se considera su incertidumbre. Por ejemplo si lo que medimos es la resistencia de una viga no podemos permitir medidas dudosas, aquí lo que hacemos es rechazar *de más*, queremos reducir la probabilidad de *falsos aceptados*.

## Cuestiones
4. Sabiendo que en el Servicio de Radioterapia considerado se ha tomado el criterio de definir el nivel de acción mayor que la tolerancia responder razonadamente si es necesario ajustar las ganancias o no a la vista de los resultados obtenidos.

*Nota*: En esta cuestión se valorará positivamente que cada alumno compare sus resultados con otros compañeros de clase (idealmente con todos) y que discutan qué decisión toman sobre las ganancias.

5. Comparar las desviaciones estándar de nuestras medidas con la estimación de incertidumbres realizada en la cuestión 3. Si son diferentes comentar cómo se justifican esas diferencias.

### Simulaciones

#### Teorema del límite central
Las funciones del módulo `simcaliblinac` están preparadas para cambiar el tamaño de la muestra de medidas y repetir la toma de muestras tantas veces como queramos. El tamaño de la muestra se indica pasando el parámetro **n** y el número de experimentos pasando el parámetro **trials**.

Ejecutando la siguiente celda obtenemos treinta muestras diferentes de cinco medidas cada una cuando fijamos 200 UM y lo almacenamos en el array de numpy muestras. Como resultado de la celda mostramos las dimensiones del array.

In [ ]:
sample_size=5
trial_number=30
muestras = scl.lectura_medida_f(UM=UM_nominales, n=sample_size, trials=trial_number)
muestras.shape

Si queremos mostrar el contenido de muestras ejecutamos esta celda

In [ ]:
muestras

Podemos calcular estádisticas del total de la muestra

In [ ]:
muestras.mean(), muestras.std()

Podemos construir una colección con las medias y desviaciones estándar de cada experimento

In [ ]:
muestras_distribucion_medias = muestras.mean(axis=1) 
muestras_distribucion_sigmas = muestras.std(axis=1, ddof=1)

# Mostrar las muestras de ambas ditribuciones
muestras_distribucion_medias, muestras_distribucion_sigmas

## Cuestión
6. Mostrar numéricamente que se verifica el teorema del límite central, es decir, que la media de las muestras de la distribución de medias es la misma que la media de todas las medidas y que dentro de incertidumbres muestrales el valor esperado de la sigma de la distribución de medias es la sigma de la distrubución total dividida por la raíz cuadrada del tamaño de la muestra

## Cuestiones

7. Hacer una gráfica de la razón entre la desviación estándar de la distribución de medias y la sigma de la distrubución total en función del número de experimientos. Discutir si tiene un comportamiento asintótico y en su caso indicar a qué valor tiende.

**Sugerencia**: Juntar el código mostrado en las celdas anteriores para definir una función dependiente del número de experimentos que devuelva la razón entre la media de la distrubución de sigmas y la sigma de la distrubución total. Generar un rango de número de experimentos, pasárselo a la función y hacer la gráfica.

8. Hacer una gráfica de la razón entre la media de la distrubución de sigmas y la sigma de la distrubución total en función del tamaño de la muestra. Analizar la importancia del parametro `ddof` para evitar el sesgo por el tamaño de la muestra. Ver la documentación de la función [numpy.std](https://numpy.org/doc/stable/reference/generated/numpy.std.html)

**Sugerencia**: Juntar el código mostrado en las celdas anteriores para definir una función dependiente del tamaño de la muestra que devuelva la razón pedida.

### Cálculo de incertidumbre
Una estimación realista de incertidumbres necesita en general incorporar el conocimiento del experimentador sobre la reproducibilidad de sus aparatos de medida. Son lo que se conoce como **incertidumbres de tipo B** que permiten estimar las sigmas de las distribuciones que siguen los elementos que participan en la medida.

La propagación de estas incertidumbres se puede hacer mediante aproximaciones analíticas, pero en casos complejos estos cálculos se pueden ver limitados por emplear solo aproximaciones de primer orden y por ignorar las covarianzas entre variables.

Una alternativa es emplear técnicas de Monte Carlo. Se pueden muestrear los valores de los elementos que intervienen en la medida si se conocen las distribuciones que siguen y se estiman sus sigmas. Combinando estos valores muestreados mediante la expresión física de la magnitud medida obtenemos una muestra de medidas cuya sigma es la incertidumbre que queremos determinar.

## Cuestión.

9. Hacer una estimación de incertidumbres mediante técnicas de Monte Carlo

**Sugerencia**:  Utilizar las funciones del módulo `simcaliblinac` para hacer experimentos de calibración en los que se obtiene en cada uno de ellos una medida de fugas, una medida de presión y temperatura y la correspondiente muestra de lecturas en el acelerador.